In [1]:
import sklearn
from sklearn.model_selection import KFold


In [12]:
import pandas as pd
chromo_sizes="/oak/stanford/groups/akundaje/ziwei75/atac_seq_pipeline/mm10/mm10_no_alt.chrom.sizes.tsv"
chromo_sizes=open(chromo_sizes).readlines()

In [17]:
chromo_list = []
for r in chromo_sizes:
    r = r.split("\t")
    if 'random' in r[0] or 'chrUn' in r[0]:
        continue
    else:
        chromo_list += [r[0]]

In [44]:
import random
fold_splits = []
for train, val in KFold(shuffle=True,random_state=42).split(chromo_list):
    train_val_chromosomes = [chromo_list[i] for i in train]
    test_chromosomes = [chromo_list[i] for i in val]
    random.shuffle(train_val_chromosomes)
    
    length = len(train_val_chromosomes)
    train_length = int(0.8 * length)
    train_chromosomes = train_val_chromosomes[:train_length]
    val_chromosomes = train_val_chromosomes[train_length:]

    
    fold_splits += [(train_chromosomes,val_chromosomes,test_chromosomes)]

In [47]:
import json
for i in range(len(fold_splits)):
    fold_dict = {"train":fold_splits[i][0],\
                "valid":fold_splits[i][1],\
                "test":fold_splits[i][2]}
    json_object = json.dumps(fold_dict, indent=4)
    
    
    # Writing to sample.json
    with open("/oak/stanford/groups/akundaje/ziwei75/ALS_colab/folds/fold_%s.json"%i, "w") as outfile:
        outfile.write(json_object)

In [50]:
test_chromsomes = []
for fold in fold_splits:
    test_chromsomes += fold[2]
    

In [52]:
len(test_chromsomes) == len(set(test_chromsomes))

True

In [54]:
len(test_chromsomes) == len(chromo_list)

True

In [57]:

for fold in fold_splits:
    train = fold[0]
    val = fold[1]
    test = fold[2]
    
    for t in train:
        assert t not in val
        assert t not in test
        
    for v in val:
        assert v not in train
        assert v not in test
        
    for t in test:
        assert t not in train
        assert t not in val